In [20]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Item 27 Use Comprehension Instead of map and filter

In [2]:
a = [1,2,3,4,5,6,7,8,9]
squares = []
for x in a:
    squares.append(x**2)
squares

[1, 4, 9, 16, 25, 36, 49, 64, 81]

In [3]:
squares = [x ** 2 for x in a]
squares

[1, 4, 9, 16, 25, 36, 49, 64, 81]

In [9]:
# the built-in map function requires the creation of a lambda function for the computation, which is visually noisy.

squares = map(lambda x: x ** 2, a)
print(list(squares))

[1, 4, 9, 16, 25, 36, 49, 64, 81]


In [6]:
# filter by comprehension
even_squares = [x ** 2 for x in a if x % 2 == 0]
even_squares

[4, 16, 36, 64]

In [8]:
# the built-in filter function is much harder to read.
alt = map(lambda x: x ** 2, filter(lambda x: x % 2 == 0, a))
assert even_squares == list(alt)

In [10]:
even_squares_dict = {x: x ** 2 for x in a if x % 2 == 0}
threes_cubed_set = {x ** 3 for x in a if x % 3 == 0}
even_squares_dict
threes_cubed_set

{2: 4, 4: 16, 6: 36, 8: 64}

{27, 216, 729}

In [13]:
alt_dict = dict(map(lambda x: (x, x ** 2),
                   filter(lambda x: x % 2 == 0, a)))
print(alt_dict)

alt_set = set(map(lambda x: x ** 3,
                  filter(lambda x: x % 3 == 0, a)))
print(alt_set)

{2: 4, 4: 16, 6: 36, 8: 64}
{216, 729, 27}


**Things to remember**

- *List Comprehensions* are clearer than the *map* and *filter* built-in functions because they don't require *lambda* expressions.

- *List Comprehensions* allow you to easily skip items from the input list, a behavior that map doesn't support without help from *filter*.


### Item 28 Avoid More than **Two** Control subexpressions in comprehensions

In [14]:
# a list comprehensions with two for subexpressions
matrix = [[1,2,3], [4,5,6], [7,8,9]]
flat = [x for row in matrix for x in row]
flat

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [15]:
squared = [[x ** 2 for x in row] for row in matrix]
squared

[[1, 4, 9], [16, 25, 36], [49, 64, 81]]

In [16]:
my_lists = [
    [[1,2,3], [4,5,6]],
    [[7,8,9], [10,11,12]],
    [[13,14,15], [16,17,18]],
]

flat = [x for sublist1 in my_lists
       for sublist2 in sublist1
       for x in sublist2]
flat

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [18]:
flat = []
for sublist1 in my_lists:
    for sublist2 in sublist1:
        flat.extend(sublist2)    # not append()
flat

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15], [16, 17, 18]]

In [20]:
# Multiple conditions at the same loop level have an implicit and expression
a = [1,2,3,4,5,6,7,8,9]
b = [x ** 2 for x in a if x > 4 if x % 2 == 0]
c = [x ** 3 for x in a if x > 4 if x % 3 == 0]
b
c 

[36, 64]

[216, 729]

In [21]:
# Conditions can be specified at each level of looping after the for subexpression
matrix = [[1,2,3], [4,5,6], [7,8,9]]
filtered = [[x for x in row if x % 3 == 0]
           for row in matrix if sum(row) >= 10]
filtered

[[6], [9]]

### Item 29 Avoid Repeated Work in Comprehensions by Using Assignment Expressions

A common pattern with comprehensions—including list, dict, and set variants—is the need to reference the same computation in multiple places.

In [6]:
stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24,
}

order = ['screws', 'wingnuts', 'clips']

def get_butches(count, size):
    return count // size

result = {}

for name in order:
    count = stock.get(name, 0)
    batches = get_butches(count, 8)
    
    if batches:
        result[name] = batches
result

{'screws': 4, 'wingnuts': 1}

In [10]:
# use dict comprehensions instead of loop
stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24,
}

order = ['screws', 'wingnuts', 'clips']

def get_butches(count, size):
    return count // size

result = {name: get_butches(stock.get(name,0), 8)
         for name in order
         if get_butches(stock.get(name, 0),8)}
result

{'screws': 4, 'wingnuts': 1}

In [12]:
# the problem with it is that the get_butches(stock.get(name, 0), 8) expression is repeated
# An easy solution to these problems is to use the walrus operator (:=), which was introduced in Python 3.8, to form an assignment expression
# as part of the comprehension

stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24,
}

order = ['screws', 'wingnuts', 'clips']

def get_batches(count, size):
    return count // size

result = {name: batches for name in order
         if (batches := get_batches(stock.get(name,0),8))}

result

{'screws': 4, 'wingnuts': 1}

In [15]:
result = {name: tenth for name, count in stock.items()
         if (tenth := count // 10)> 0}
result

{'nails': 12, 'screws': 3, 'washers': 2}

In [16]:
# If a comprehension uses the walrus operator in the value part of the comprehension and doesn’t have a condition, 
# it’ll leak the loop variable into the containing scope
half = [(last := count // 2) for count in stock.values()]
print(f'Last item of {half} is {last}')

Last item of [62, 17, 4, 12] is 12


In [18]:
# same as a normal for loop 
for count in stock.values():    # leaks loop variable
    pass

print(f'Last item of {list(stock.values())} is {count}')


Last item of [125, 35, 8, 24] is 24


In [22]:
# However, similar leakage doesn't happen for the loop variables from comprehensions
half = [count2 // 2 for count2 in stock.values()]
print(half)
print(count2)

[62, 17, 4, 12]


NameError: name 'count2' is not defined

**THings to remember**

- It’s better not to leak loop variables, so I recommend using assignment
expressions only in the condition part of a comprehension.

In [25]:
# Using an assignment expression also works the same way in generator expressions.
stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24,
}

order = ['screws', 'wingnuts', 'clips']

def get_batches(count, size):
    return count // size

found = ((name, batches) for name in order
        if (batches := get_batches(stock.get(name, 0),8)))
next(found)
next(found)
next(found)

('screws', 4)

('wingnuts', 1)

StopIteration: 